# Applying k-Fold Cross validation to select DNA Transcription Factor Prediction Model

k-fold cross validation is applied to existing K- Nearest neighbour regression model for the DNA Transcription Factor (TF) prediction.

In [51]:
import pandas as pd 
import csv
import sys
import numpy as np
import decimal 

pd.set_option('display.max_colwidth', -1)

## Importing the files
Importing the TF_sequence.txt and TF_output.txt files.


In [3]:
X_train = pd.read_csv("TF_sequences.txt",sep='\t', lineterminator='\n',header=None)
X_train.columns = ["TF_Name","TF_Sequence"]
#X_train["L"] = X_train["TF_Sequence"].str.len()
X_train

,TF_Name,TF_Sequence
0,Alx3,RRNRTTFSTFQLEELEKVFQKTHYPDVYAREQLALRTDLTEARVQVWFQNRRAKWRK
1,Alx4,RRNRTTFTSYQLEELEKVFQKTHYPDVYAREQLAMRTDLTEARVQVWFQNRRAKWRK
2,Arx,RRYRTTFTSYQLEELERAFQKTHYPDVFTREELAMRLDLTEARVQVWFQNRRAKWRK
3,Bapx1,KRSRAAFSHAQVFELERRFNHQRYLSGPERADLAASLKLTETQVKIWFQNRRYKTKR
4,Barhl1,RKARTAFTDHQLAQLERSFERQKYLSVQDRMELAASLNLTDTQVKTWYQNRRTKWKR
...,...,...
145,Tlx2,KKPRTSFSRSQVLELERRFLRQKYLASAERAALAKALRMTDAQVKTWFQNRRTKWRR
146,Uncx4.1,RRTRTNFTGWQLEELEKAFNESHYPDVFMREALALRLDLVESRVQVWFQNRRAKWRK
147,Vax1,KRTRTSFTAEQLYRLEMEFQRCQYVVGRERTELARQLNLSETQVKVWFQNRRTKQKK
148,Vax2,KRTRTSFTAEQLYRLEMEFQRCQYVVGRERTELARQLNLSETQVKVWFQNRRTKQKK


In [4]:
Y_train = pd.read_csv('TF_output.txt',sep='\t', lineterminator = '\n')
Y_train

,Alx3,Alx4,Arx,Bapx1,Barhl1,Barhl2,Barx1,Barx2,Bsx,Cart1,...,Tcf1,Tcf2,Tgif1,Tgif2,Titf1,Tlx2,Uncx4.1,Vax1,Vax2,Vsx1
0,2.3401,1.7044,2.5305,0.7962,1.8823,3.2222,1.2282,2.0546,0.5882,2.2736,...,1.4632,-0.0636,1.2235,0.0742,2.0043,0.5475,1.3708,1.6994,2.2566,3.3771
1,1.3416,1.4467,0.9565,0.0567,1.8291,2.4403,0.9911,1.2143,0.2469,1.2506,...,1.1287,1.0444,-0.2575,0.9416,0.3597,-0.2338,0.9116,1.2902,0.9958,1.2923
2,1.3386,1.0929,1.4749,1.1892,1.0593,1.4076,0.3329,1.2362,0.0150,2.1433,...,0.5170,0.4963,0.3452,0.2785,2.1461,-0.3770,0.3261,1.0687,0.3439,2.3979
3,2.1334,2.3737,1.9087,0.2495,2.1892,2.2676,2.2716,2.1577,1.1156,2.5319,...,1.5256,0.6547,1.6134,-0.0660,1.3260,0.2647,2.1307,1.6622,1.9085,3.1186
4,1.6949,1.1549,0.5943,-0.2069,1.3848,2.5651,1.5693,1.9802,0.1341,2.0363,...,1.3839,0.8750,0.1880,0.7217,1.2139,0.5259,0.8990,1.4452,1.4360,-0.0793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32891,0.5581,0.2975,0.5163,-0.0735,1.1483,0.6355,0.4882,0.9772,0.6425,0.4395,...,0.8217,0.3721,0.4734,-0.4811,0.5929,1.2897,0.4742,0.4387,0.3621,1.4778
32892,0.8202,-0.0643,-0.5583,0.0615,0.5337,1.0115,-0.1255,0.3753,-0.0101,0.9619,...,-0.1780,0.2357,5.2360,0.5475,0.9197,1.4513,0.6367,0.5879,-0.0045,-0.8305
32893,1.6207,1.2021,1.7095,1.7572,2.3076,2.1069,0.9452,1.7782,0.9299,2.2271,...,1.4767,0.6399,2.3455,1.1353,2.6903,1.8336,1.5573,1.2393,1.7834,1.8334
32894,1.0773,-0.3050,-0.5452,0.8667,2.0427,0.1398,0.7705,0.5219,1.1298,0.1653,...,2.2999,0.1311,-1.5313,-0.2867,-1.5910,1.5867,1.8107,2.1038,0.0216,1.3018


### Preparing the input file 
Writing the files into two files named **X_train.txt** and **Y_train.txt** for command line input for the python program.

In [5]:
X_train.to_csv("X_train.txt", sep="\t",index=False,header=False)

In [6]:
Y_train.to_csv("Y_train.txt", sep="\t",index=False,float_format='%.4f')

### Reading the file 
Assuming files are received from command line and reading the file in **X_train** and **Y_train** dataframe.

- Changing column name for the X_train dataframe

In [7]:
X_train = pd.read_csv('X_train.txt',sep='\t',header=None)
X_train.columns = ['TF_Name','TF_Seq']
X_train

,TF_Name,TF_Seq
0,Alx3,RRNRTTFSTFQLEELEKVFQKTHYPDVYAREQLALRTDLTEARVQVWFQNRRAKWRK
1,Alx4,RRNRTTFTSYQLEELEKVFQKTHYPDVYAREQLAMRTDLTEARVQVWFQNRRAKWRK
2,Arx,RRYRTTFTSYQLEELERAFQKTHYPDVFTREELAMRLDLTEARVQVWFQNRRAKWRK
3,Bapx1,KRSRAAFSHAQVFELERRFNHQRYLSGPERADLAASLKLTETQVKIWFQNRRYKTKR
4,Barhl1,RKARTAFTDHQLAQLERSFERQKYLSVQDRMELAASLNLTDTQVKTWYQNRRTKWKR
...,...,...
145,Tlx2,KKPRTSFSRSQVLELERRFLRQKYLASAERAALAKALRMTDAQVKTWFQNRRTKWRR
146,Uncx4.1,RRTRTNFTGWQLEELEKAFNESHYPDVFMREALALRLDLVESRVQVWFQNRRAKWRK
147,Vax1,KRTRTSFTAEQLYRLEMEFQRCQYVVGRERTELARQLNLSETQVKVWFQNRRTKQKK
148,Vax2,KRTRTSFTAEQLYRLEMEFQRCQYVVGRERTELARQLNLSETQVKVWFQNRRTKQKK


In [8]:
Y_train = pd.read_csv('Y_train.txt',sep='\t')
Y_train

,Alx3,Alx4,Arx,Bapx1,Barhl1,Barhl2,Barx1,Barx2,Bsx,Cart1,...,Tcf1,Tcf2,Tgif1,Tgif2,Titf1,Tlx2,Uncx4.1,Vax1,Vax2,Vsx1
0,2.3401,1.7044,2.5305,0.7962,1.8823,3.2222,1.2282,2.0546,0.5882,2.2736,...,1.4632,-0.0636,1.2235,0.0742,2.0043,0.5475,1.3708,1.6994,2.2566,3.3771
1,1.3416,1.4467,0.9565,0.0567,1.8291,2.4403,0.9911,1.2143,0.2469,1.2506,...,1.1287,1.0444,-0.2575,0.9416,0.3597,-0.2338,0.9116,1.2902,0.9958,1.2923
2,1.3386,1.0929,1.4749,1.1892,1.0593,1.4076,0.3329,1.2362,0.0150,2.1433,...,0.5170,0.4963,0.3452,0.2785,2.1461,-0.3770,0.3261,1.0687,0.3439,2.3979
3,2.1334,2.3737,1.9087,0.2495,2.1892,2.2676,2.2716,2.1577,1.1156,2.5319,...,1.5256,0.6547,1.6134,-0.0660,1.3260,0.2647,2.1307,1.6622,1.9085,3.1186
4,1.6949,1.1549,0.5943,-0.2069,1.3848,2.5651,1.5693,1.9802,0.1341,2.0363,...,1.3839,0.8750,0.1880,0.7217,1.2139,0.5259,0.8990,1.4452,1.4360,-0.0793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32891,0.5581,0.2975,0.5163,-0.0735,1.1483,0.6355,0.4882,0.9772,0.6425,0.4395,...,0.8217,0.3721,0.4734,-0.4811,0.5929,1.2897,0.4742,0.4387,0.3621,1.4778
32892,0.8202,-0.0643,-0.5583,0.0615,0.5337,1.0115,-0.1255,0.3753,-0.0101,0.9619,...,-0.1780,0.2357,5.2360,0.5475,0.9197,1.4513,0.6367,0.5879,-0.0045,-0.8305
32893,1.6207,1.2021,1.7095,1.7572,2.3076,2.1069,0.9452,1.7782,0.9299,2.2271,...,1.4767,0.6399,2.3455,1.1353,2.6903,1.8336,1.5573,1.2393,1.7834,1.8334
32894,1.0773,-0.3050,-0.5452,0.8667,2.0427,0.1398,0.7705,0.5219,1.1298,0.1653,...,2.2999,0.1311,-1.5313,-0.2867,-1.5910,1.5867,1.8107,2.1038,0.0216,1.3018


#### Importing KFold from scikit learn library

In [9]:
from sklearn.model_selection import KFold

Testing the KFold spliiter and the iteration.

In [10]:
cv = KFold(n_splits=5)
for train_index,test_index in cv.split(X_train):
    print("Train Index:", train_index,"\n")
    print("Test Index:", test_index)
    

Train Index: [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] 

Test Index: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
Train Index: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 1

Defining distance function 1 (**D1**)

In [11]:
def CalculateDistance1(aminoAcidSeq1, aminoAcidSeq2):
    #print(aminoAcidSeq1,aminoAcidSeq2)
    distance = 0
    try:
        if aminoAcidSeq1[2] != aminoAcidSeq2[2]:
            distance+=1
        if aminoAcidSeq1[4] != aminoAcidSeq2[4]:
            distance+=1
        if aminoAcidSeq1[5] != aminoAcidSeq2[5]:
            distance+=1
        if aminoAcidSeq1[24] != aminoAcidSeq2[24]:
            distance+=1
        if aminoAcidSeq1[30] != aminoAcidSeq2[30]:
            distance+=1
        if aminoAcidSeq1[43] != aminoAcidSeq2[43]:
            distance+=1
        if aminoAcidSeq1[45] != aminoAcidSeq2[45]:
            distance+=1
        if aminoAcidSeq1[46] != aminoAcidSeq2[46]:
            distance+=1
        if aminoAcidSeq1[47] != aminoAcidSeq2[47]:
            distance+=1
        if aminoAcidSeq1[49] != aminoAcidSeq2[49]:
            distance+=1
        if aminoAcidSeq1[50] != aminoAcidSeq2[50]:
            distance+=1
        if aminoAcidSeq1[52] != aminoAcidSeq2[52]:
            distance+=1
        if aminoAcidSeq1[53] != aminoAcidSeq2[53]:
            distance+=1
        if (len(aminoAcidSeq1)>54 & len(aminoAcidSeq2)>54):
            if aminoAcidSeq1[54] != aminoAcidSeq2[54]:
                distance+=1
        if (len(aminoAcidSeq1)>56 & len(aminoAcidSeq2)>56):
            if (aminoAcidSeq1[56] != aminoAcidSeq2[56]):
                distance+=1
    except IndexError:
        print('Amino ACID 1: ',aminoAcidSeq1,' length: '+str(len(aminoAcidSeq1)))
        print('Amino ACID 2: ',aminoAcidSeq2,' length: '+str(len(aminoAcidSeq2)))
    return distance

### Distance Function 2
using Jaccard String Similarity algorithm

In [33]:
def CalculateDistance2(aminoAcidSeq1,aminoAcidSeq2):
    intersection_cardinality = len(set.intersection(*[set(aminoAcidSeq1),set(aminoAcidSeq2)]))
    union_cardinality = len(set.union(*[set(aminoAcidSeq1),set(aminoAcidSeq2)]))
    return intersection_cardinality/float(union_cardinality)

#print (CalculateDistance2("RKARTAFSDHQLNQLERSFERQKYLSVQDRMDLAAALNLTDTQVKTWYQNRRTKWKR","RRNRFKWGPASQQILFQAYERQKSPSKEERETLVEECNRAECIQRGVSPSQAQGLGSNLVTEVRVYNWFANRRKEEAF"))



0.65


## Grid Search Function definition
Definition for function that searches on a given dataframe to select the best model on basis of performance values

In [ ]:
def GridSearch():
    

## Model iterating for K = [ 7, 8, 9 ] and 5-fold Cross Validation using two distance function
Preparing the train-test pipeline for one distance function (old one) and evaluating performance.

In [66]:
import scipy.stats as stats
cv = KFold(n_splits=5)
performance_values = pd.DataFrame(columns=['standard_deviationD1','mean_spearmans_corelD1','standard_deviationD2','mean_spearmans_corelD2'])
Y_predicted = pd.DataFrame()

performance_grid = pd.DataFrame(columns=['k','D1','D2'])
k_values = [7,8,9]

for k in k_values:
    for train_index,test_index in cv.split(X_train):
        spearmans_corel_d1= 0
        standard_deviation_d1=0
        spearmans_corel_d2 = 0
        standard_deviation_d2 = 0
        for i in test_index:
            X_train_CV = X_train.loc[train_index,:]
            X_test_CV = X_train.loc[test_index,:]
            # model using distance function 1
            #X_train_CV
            X_train_CV['Distance1'] = X_train.apply(lambda row: CalculateDistance1(row['TF_Seq'],X_test_CV.loc[i]['TF_Seq']),axis=1)
            X_train_CV = X_train_CV.sort_values(by='Distance1')
            neighbours = X_train_CV.head(k)['TF_Name']
            # select the output vectors from the Y_train dataframe
            neighbour_vectors = Y_train[neighbours.tolist()]
            neighbour_vectors[X_test_CV.loc[i]['TF_Name'] + '(Predicted D1)']= neighbour_vectors.mean(axis=1)
            neighbour_vectors[X_test_CV.loc[i]['TF_Name']] = Y_train[X_test_CV.loc[i]['TF_Name']]
            #print(neighbour_vectors)
            #Y_predicted[neighbour_vectors[X_test_CV.loc[i]['TF_Name']]] = neighbour_vectors[X_test_CV.loc[i]['TF_Name'] + '(Predicted D1)']
            mean_std_predicted_values = neighbour_vectors.loc[:, neighbour_vectors.columns != X_test_CV.loc[i]['TF_Name']].std(axis=1)
            #standard_deviation_d1+= np.std(neighbour_vectors[X_test_CV.loc[i]['TF_Name']])
            standard_deviation_d1+= mean_std_predicted_values.mean() 
            #print (mean_std_predicted_values.mean())

            spearmans_corel_d1 += stats.spearmanr(neighbour_vectors[X_test_CV.loc[i]['TF_Name']],neighbour_vectors[X_test_CV.loc[i]['TF_Name'] + '(Predicted D1)'])[0]
            ############################################################################################33
            # Removing distance1 column and adding distance2 column and sorting
            X_train_CV.drop(columns=['Distance1'],inplace=True)
            X_train_CV['Distance2'] = X_train.apply(lambda row: CalculateDistance2(row['TF_Seq'],X_test_CV.loc[i]['TF_Seq']),axis=1)
            X_train_CV = X_train_CV.sort_values(by='Distance2')
            neighbours = X_train_CV.head(k)['TF_Name']
            neighbour_vectors = Y_train[neighbours.tolist()]
            neighbour_vectors[X_test_CV.loc[i]['TF_Name'] + '(Predicted D2)']= neighbour_vectors.mean(axis=1)
            neighbour_vectors[X_test_CV.loc[i]['TF_Name']] = Y_train[X_test_CV.loc[i]['TF_Name']]
            mean_std_predicted_values = neighbour_vectors.loc[:, neighbour_vectors.columns != X_test_CV.loc[i]['TF_Name']].std(axis=1)

            standard_deviation_d2+= mean_std_predicted_values.mean() 
            spearmans_corel_d2 += stats.spearmanr(neighbour_vectors[X_test_CV.loc[i]['TF_Name']],neighbour_vectors[X_test_CV.loc[i]['TF_Name'] + '(Predicted D2)'])[0]

            #print(neighbour_vectors)

        mean_standard_deviation_d1 = standard_deviation_d1/test_index.size
        mean_standard_deviation_d2 = standard_deviation_d2/test_index.size
        #mean_standard_deviation_d1    #print (Y_predicted)

        mean_spearman_cor_d1 = spearmans_corel_d1/test_index.size
        mean_spearman_cor_d2 = spearmans_corel_d2/test_index.size

        performance_row =  pd.DataFrame([[mean_standard_deviation_d1, mean_spearman_cor_d1,mean_standard_deviation_d2,mean_spearman_cor_d2]],columns=['standard_deviationD1','mean_spearmans_corelD1','standard_deviationD2','mean_spearmans_corelD2'])
        performance_values = performance_values.append(performance_row, ignore_index = True)

        print('Mean Spearman COR :',mean_spearman_cor_d1) 

    #print('Spearman COR:')
    print('Performance Dataframe:\n', performance_values)
    meanSpearmand1 = round(performance_values['mean_spearmans_corelD1'].mean(),4)
    meanSpearmand2 = round(performance_values['mean_spearmans_corelD2'].mean(),4)

    meanSD1 = round(performance_values['standard_deviationD1'].mean(),4)
    meanSD2 = round(performance_values['standard_deviationD2'].mean(),4)

    #print(test)

    performance_grid_row =  pd.DataFrame([[k,str(meanSpearmand1) +' ± '+ str(meanSD1),str(meanSpearmand2)+'±'+str(meanSD2)]],columns=['k','D1','D2'])
    performance_grid = performance_grid.append(performance_grid_row,ignore_index = True)

    print('Performance Grid \n')
    print(performance_grid)



C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

Mean Spearman COR : 0.826653061696645
Mean Spearman COR : 0.8410400581802416
Mean Spearman COR : 0.8112372087782457
Mean Spearman COR : 0.7239758394894417
Mean Spearman COR : 0.7469519750762453
Performance Dataframe:
    standard_deviationD1  mean_spearmans_corelD1  standard_deviationD2  \
0  0.503304              0.826653                0.477178               
1  0.474164              0.841040                0.549178               
2  0.493411              0.811237                0.514902               
3  0.588456              0.723976                0.481116               
4  0.568191              0.746952                0.467285               

   mean_spearmans_corelD2  
0  0.803986                
1  0.813860                
2  0.813478                
3  0.656755                
4  0.710190                
0.79
Performance Grid 

   k             D1             D2
0  7  0.79 ± 0.5255  0.7597±0.4979
Mean Spearman COR : 0.8291671383404935
Mean Spearman COR : 0.8439979887586572
Mea